# Cifrado simétrico

Las prácticas de este curso las haremos en Python, conde tenemos dos posibilidades para probar criptografía: el paquete `PyCryptodome` y el paquete `cryptography`. Ambos son opciones válidas e intercambiables. Las prácticas de este curso las haremos con PyCryptodome. Puedes encontrar la ayuda en: https://pycryptodome.readthedocs.io/en/latest/

Si estás siguiendo estas notas en tu PC, puedes instalarlo con: `python3 -m pip install pycryptodome`. Si las estás siguiendo en Colab, ejecuta la siguiente línea:

In [ ]:
!pip install pycryptodome

## Cifrado de flujo: ChaCha20

En esta sección veremos los comandos para enviar un texto cifrado con ChaCha20.

Primero importamos las librerías que necesitamos:

In [ ]:
from base64 import b64encode, b64decode
from Crypto.Cipher import ChaCha20
import Crypto.Random

### Configuración del sistema emisor

Los módulos de criptografía suelen necesitar una etapa inicial de configuración. Cada módulo se configura a su manera. A continuación encontrarás la etapa de configuración de ChaCha20 para PyCryptodome.

Fíjate que la clave se crea al azar con algoritmos criptográficos `Crypto.Random.get_random_bytes()`: **es fundamental que las claves sean totalmente aleatorias y creadas también con algoritmos criptográficos**. No todas las funciones de creación de azar son válidas: necesitas una fuente de azar con validez criptográfica. Muchas implementaciones de protocolos criptográficos han caído no porque el cifrado fuese erróneo, sino porque la fuente de azar no era lo suficiente "aleatoria". En particular: en criptografía, no uses la función general `random.randbytes()` sino alguna específica de la librería criptográfica, como `Crypto.Random.get_random_bytes()`. Esto es válido también para los demás sistemas criptográficos.

In [ ]:
key = Crypto.Random.get_random_bytes(32)
cipher_emisor = ChaCha20.new(key=key, nonce=None)
print(f'Longitud de la clave: {8 * len(key)} bits')

En PyCryptodome el *nonce* se puede pasar al algoritmo durante la configuración. Si, como en este caso, no se pasa *nonce* durante la creación, la librería crea un *nonce* al azar que podemos recuperar. Si decides crear tú el *nonce*, recuerda que también tiene que ser un número aleatorio creado con algoritmos criptográficos, igual que la clave.

Observa: en ChaCha20, la clave tiene 256 bits y el nonce tiene 64 bits.

In [ ]:
nonce = cipher_emisor.nonce
print(f'nonce creado automáticamente: {b64encode(nonce)}, longitud: {8 * len(nonce)} bits')

# si lo necesitas:
# nonce_creado_por_mi = Crypto.Random.get_random_bytes(8)
# cipher_emisor = ChaCha20.new(key=key, nonce=once_creado_por_mi)

### Cifrado del mensaje por el emisor

El emisor cifra el mensaje `Atacaremos al amanecer` y envía al receptor `result`, es decir, la pareja "mensaje cifrado" y "*nonce*". Fíjate: el nonce se puede enviar por un canal inseguro, así que se asume que el atacante lo conocerá.

Observa que el resultado lo codificamos en Base64 (https://es.wikipedia.org/wiki/Base64). Aunque no es necesario, sí que es común hacerlo así porque algunos protocolos (correo electrónico, JSON...) solo puede enviar caracteres imprimibles. No pierdes ni ganas seguridad si decides usar o no Base64, es más una exigencia de tu sistema de comunicaciones. Fíjate que he usado la expresión "codificamos en Base64", no ciframos. Base64 es un algoritmo de codificación de bytes, no tiene claves, cualquiera lo puede codificar y decodificar y por tanto Base64 no es un cifrado.

In [ ]:
plaintext = b'Atacaremos al amanecer'
ciphertext = cipher_emisor.encrypt(plaintext)
ct = b64encode(ciphertext)
result = {'nonce':b64encode(nonce), 'ciphertext':ct}
print(result)

### Recepción y descifrado

El receptor toma el *nonce* y el *ciphertext*. Primero decodifica el base64, configura el *cipher* y el *nonce* que ha recibido y descifra. Ya veremos cómo el receptor conoce la clave, porque no se la puede enviar el emisor.

In [ ]:
received_nonce = b64decode(result['nonce'])
received_ciphertext = b64decode(result['ciphertext'])
cipher_receptor = ChaCha20.new(key=key, nonce=received_nonce)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

### Siguientes mensajes: sincronización entre ciphers

Supongamos que el usuario vuelve a enviar el mismo mensaje, con el mismo cipher (fíjate que no volvemos a definir `cipher_emisor`: lo estamos reaprovechando)

In [ ]:
plaintext = b'Atacaremos al amanecer'
ciphertext = cipher_emisor.encrypt(plaintext)
ct = b64encode(ciphertext)
result = {'nonce':b64encode(nonce), 'ciphertext':ct}
print(result)

Fíjate: estamos cifrando el mismo mensaje con el mismo nonce... pero el ciphertext es diferente. ¿Recuerdas que nunca se debe cifrar el mismo texto con la misma clave? ChaCha20 nos ayuda a que no lo hagamos, ni siquiera por equivocación, mediante el uso de un contador.

Supongamos que el receptor crea un nuevo cipher, con la misma configuración de key y nonce, e intenta descifrar:

In [ ]:
received_nonce = b64decode(result['nonce'])
received_ciphertext = b64decode(result['ciphertext'])
cipher_receptor = ChaCha20.new(key=key, nonce=received_nonce)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

¿Qué ha pasado? ¿Por qué no se descifra? Recuerda que ChaCha20 tiene un contador adicional interno. **Emisor y receptor tienen que estar sincronizados**: para descifrar el byte número 22 tenemos que decirle al receptor que han pasado 22 bytes antes, aunque no los haya visto.

(nota: 22 es el tamaño en bytes de la cadena "Atacaremos al amanecer", que fue el contenido del primer mensaje)

Si volvemos a intentar descifrar, ahora sí que podemos hacerlo:

In [ ]:
cipher_receptor.seek(22)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

PyCryptodome y todos los demás están sincronizados siempre que descifremos los mismos bytes que hemos cifrado desde que se han creado los dos ciphers, el de emisión y el de recepción.

Si alguno de los dos pierde la sincronización (por ejemplo, porque se reinicia), entonces es necesario volver a sincronizarlos con un "seek": "ya envié XX bytes aunque no los hayas visto, mueve el estado a esta posición"

Poder volver a sincronizar los dos streams es una enorme ventaja de ChaCha20 y eso es por el parámetro `pos` autoincremental que forma parte de la matriz de estado. No todos los algoritmos permiten sincronizar los flujos si se pierde la sincronización.

## Cifrado de bloque: modos de cifrado AES

El otro sistema de cifrado simétrico que veremos es AES. Como ya comentamos en la parte teórica, no es suficiente con indicar que usamos AES, también es necesario especificar de qué manera, es decir, **en qué modo estamos usando AES**.

Vamos a crear:

- un mensaje de 128 bits, el tamaño de bloque de AES.
- una clave k de 128 bits que usaremos durante todo el ejercicio.

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

m = b'Atacad a las dos'
k = get_random_bytes(16)
print(f'Mensaje: "{m}" Tamaño={len(m) * 8} bits')
print(f'Clave: {b64encode(k)} Tamaño={len(k) * 8} bits')

### Modo ECB

El primer modo que veremos es "ECB": cada bloque se cifra de forma independiente, sin realimentación con los demás.

![center w:30em](images/ECB_encryption.svg)

Vamos a ciframos tres veces el mismo mensaje. Observa que no hay memoria, y que cifrar dos veces el mismo mensaje con la misma clave produce el mismo texto cifrado. Por eso no se debe usar nunca el modo ECB: un atacante que observa las comunicaciones quizá no sepa qué estamos diciendo, pero sí que sabe que estamos repitiendo el mensaje y eso puede ser suficiente para sus objetivos.

**Nunca debe usarse AES en modo ECB**

In [ ]:
cipher = AES.new(k, AES.MODE_ECB)
c1 = cipher.encrypt(m)
c2 = cipher.encrypt(m)
c3 = cipher.encrypt(m)
print(b64encode(c1))
print(b64encode(c2))
print(b64encode(c3))

In [ ]:
decipher = AES.new(k, AES.MODE_ECB)
m1 = decipher.decrypt(c1)
m2 = decipher.decrypt(c2)
m3 = decipher.decrypt(c3)
print(m1)
print(m2)
print(m3)

### Modo CBC

En el modo CBC, hay realimentación entre bloques y existe un vector de inicialización

![](images/CBC_encryption.svg)

Ciframos dos veces el mismo mensaje. Observa que tenemos que crear un IV (Vector de Inicialización), y que este IV se lo tenemos que enviar al receptor. El envío del IV se hace durante el primer mensaje, antes de que el canal sea seguro, pero no hay problemas en que un atacante conozca el IV.

In [ ]:
iv = get_random_bytes(16)
cipher = AES.new(k, AES.MODE_CBC, iv=iv)
c1 = cipher.encrypt(m)
c2 = cipher.encrypt(m)
print(b64encode(c1))
print(b64encode(c2))

***

<font color="green">
PREGUNTA: ahora los dos cifrados son diferentes a pesar de que estamos cifrando el mismo mensaje. ¿Por qué sucede eso?
</font>

***

<font color="blue">
RESPUESTA: añade aquí tu respuesta
</font>

***

Descifrado: necesita la clave y el IV. La clave es secreta y el receptor tiene que haberla recibido por un canal secreto (lo veremos en la siguiente sección) pero el IV puede recibirse sin protección al inicio de la comunicación.

In [ ]:
decipher = AES.new(k, AES.MODE_CBC, iv=iv)
m1 = decipher.decrypt(c1)
m2 = decipher.decrypt(c2)
print(m1)
print(m2)

### Padding

¿Qué pasa si tenemos que enviar mensajes más cortos que la longitud de bloque de AES? Entonces tenemos que usar algún algoritmo de padding. Es decir: marcar la longitud del mensaje original. Observa que solo podemos enviar bloques de 128 bits, y si intentamos enviar bloques más cortos o más largos saltará un error:

In [ ]:
c1 = cipher.encrypt(b'mensaje corto')

Con Cryptodome podemos usar las funciones `pad()` y `unpad()`

Observa: en este ejemplo no ponemos IV, pero ya sabes que hay un IV. Si no lo especificamos los escogerá al azar.

In [ ]:
from Crypto.Util.Padding import pad, unpad

# mensaje corto
m = b'1234'
cipher = AES.new(k, AES.MODE_CBC)
c = cipher.encrypt(pad(m, AES.block_size))
print({'iv':b64encode(cipher.iv), 'ciphertext':b64encode(c)})

Recepción:

In [ ]:
decipher = AES.new(k, AES.MODE_CBC, cipher.iv)
pt = unpad(decipher.decrypt(c), AES.block_size)
print("The message was: ", pt)

¿Qué pasa si no usamos unpad? AES es un cifrado de bloque, así que los mensajes en AES tienen obligatoriamente un tamaño igual al bloque AES (128 bits), así que vemos el *padding* que sobra. Las función *unpad()* nos hubiese cortado esos bytes sobrantes.

In [ ]:
decipher = AES.new(k, AES.MODE_CBC, cipher.iv)
pt = decipher.decrypt(c)
print(f"The message was: {pt} (longitud {len(pt) * 8} bits)")

***

<font color="red">
PREGUNTA: vimos que ChaCha20 podía resincronizarse si se perdían mensajes. ¿AES en modo CBC puede resincronizarse si se pierden mensajes? ¿Qué modo de AES permitiría resincronizarse?
</font>

***

<font color="blue">
RESPUESTA: añade aquí tu respuesta
</font>

***

# Cifrado asimétrico o de clave pública

## Acuerdo de claves Diffie-Hellman

El protocolo Diffie-Hellman (D-H) permite que dos personas que no se han visto nunca acuerden una clave que después pueden utilizar para el cifrado AES o ChaCha20. Gracias a D-H, Nadie más que esas dos personas sabrán que clave han acordado, incluso si un atacante leyese todos los mensajes que se han intercambiado desde el principio.

Recuerda, el protocolo D-H es:

1. Acuerdan $g$ y $p$ primos entre sí
1. Escogen números en secreto $a$ y $b$
1. Se envian entre ellos:
    - $Alice \rightarrow Bob: A=g^{a} \mod p$
    - $Bob \rightarrow Alice: B=g^{b} \mod p$
1. Calculan en secreto:
    - $Alice$: $s = B^{a} \mod p = g^{ab} \mod p$
    - $Alice$: $s = A^{b} \mod p = g^{ab} \mod p$
1. Y usan $s$ como clave de cifrado un algoritmo simétrico  

A continuación está el código de la librería https://github.com/amiralis/pyDH de Amirali Sanatinia, que es sencillo de leer y entender.

Aunque no parece haber errores evidentes, **es obligatorio utilizar librerías auditadas**. Seguiremos esta por su valor educativo, no porque sea recomendable su uso.

In [ ]:
"""
Pure Python Diffie Hellman implementation

Source: https://github.com/amiralis/pyDH

Apache License
         Version 2.0, January 2004
     Copyright 2015 Amirali Sanatinia
"""

import os
import binascii
import hashlib

# RFC 3526 - More Modular Exponential (MODP) Diffie-Hellman groups for 
# Internet Key Exchange (IKE) https://tools.ietf.org/html/rfc3526 

primes = {
	
	# 1536-bit
	5: { 
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA237327FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 2048-bit
	14: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 3072-bit 
	15: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A93AD2CAFFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 4096-bit
	16: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C934063199FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 6144-bit
	17: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C93402849236C3FAB4D27C7026C1D4DCB2602646DEC9751E763DBA37BDF8FF9406AD9E530EE5DB382F413001AEB06A53ED9027D831179727B0865A8918DA3EDBEBCF9B14ED44CE6CBACED4BB1BDB7F1447E6CC254B332051512BD7AF426FB8F401378CD2BF5983CA01C64B92ECF032EA15D1721D03F482D7CE6E74FEF6D55E702F46980C82B5A84031900B1C9E59E7C97FBEC7E8F323A97A7E36CC88BE0F1D45B7FF585AC54BD407B22B4154AACC8F6D7EBF48E1D814CC5ED20F8037E0A79715EEF29BE32806A1D58BB7C5DA76F550AA3D8A1FBFF0EB19CCB1A313D55CDA56C9EC2EF29632387FE8D76E3C0468043E8F663F4860EE12BF2D5B0B7474D6E694F91E6DCC4024FFFFFFFFFFFFFFFF,
	"generator": 2
	},

	# 8192-bit
	18: {
	"prime": 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C93402849236C3FAB4D27C7026C1D4DCB2602646DEC9751E763DBA37BDF8FF9406AD9E530EE5DB382F413001AEB06A53ED9027D831179727B0865A8918DA3EDBEBCF9B14ED44CE6CBACED4BB1BDB7F1447E6CC254B332051512BD7AF426FB8F401378CD2BF5983CA01C64B92ECF032EA15D1721D03F482D7CE6E74FEF6D55E702F46980C82B5A84031900B1C9E59E7C97FBEC7E8F323A97A7E36CC88BE0F1D45B7FF585AC54BD407B22B4154AACC8F6D7EBF48E1D814CC5ED20F8037E0A79715EEF29BE32806A1D58BB7C5DA76F550AA3D8A1FBFF0EB19CCB1A313D55CDA56C9EC2EF29632387FE8D76E3C0468043E8F663F4860EE12BF2D5B0B7474D6E694F91E6DBE115974A3926F12FEE5E438777CB6A932DF8CD8BEC4D073B931BA3BC832B68D9DD300741FA7BF8AFC47ED2576F6936BA424663AAB639C5AE4F5683423B4742BF1C978238F16CBE39D652DE3FDB8BEFC848AD922222E04A4037C0713EB57A81A23F0C73473FC646CEA306B4BCBC8862F8385DDFA9D4B7FA2C087E879683303ED5BDD3A062B3CF5B3A278A66D2A13F83F44F82DDF310EE074AB6A364597E899A0255DC164F31CC50846851DF9AB48195DED7EA1B1D510BD7EE74D73FAF36BC31ECFA268359046F4EB879F924009438B481C6CD7889A002ED5EE382BC9190DA6FC026E479558E4475677E9AA9E3050E2765694DFC81F56E880B96E7160C980DD98EDD3DFFFFFFFFFFFFFFFFF,
	"generator": 2
	}
}


class DiffieHellman:
	""" Class to represent the Diffie-Hellman key exchange protocol """
	# Current minimum recommendation is 2048 bit.
	def __init__(self, group=14):
		if group in primes:
			self.p = primes[group]["prime"]
			self.g = primes[group]["generator"]
		else:
			raise Exception("Group not supported")

		self.__a = int(binascii.hexlify(os.urandom(32)), base=16)

	def get_private_key(self):
		""" Return the private key (a) """
		return self.__a

	def gen_public_key(self):
		""" Return A, A = g ^ a mod p """
		# calculate G^a mod p
		return pow(self.g, self.__a, self.p)

	def check_other_public_key(self, other_contribution):
		# check if the other public key is valid based on NIST SP800-56
		# 2 <= g^b <= p-2 and Lagrange for safe primes (g^bq)=1, q=(p-1)/2

		if 2 <= other_contribution and other_contribution <= self.p - 2:
			if pow(other_contribution, (self.p - 1) // 2, self.p) == 1:
				return True
		return False

	def gen_shared_key(self, other_contribution):
		""" Return g ^ ab mod p """
		# calculate the shared key G^ab mod p
		if self.check_other_public_key(other_contribution):
			self.shared_key = pow(other_contribution, self.__a, self.p)
			return hashlib.sha256(str(self.shared_key).encode()).digest()
		else:
			raise Exception("Bad public key from other party")
            

### Alice

En esta ocasión, Alice y Bob quieren acordar una clave que después usarán para las comunicaciones con AES.

Vamos a ver primero qué hace Alice.

In [ ]:
alice = DiffieHellman()
alice_pubkey = alice.gen_public_key()

print('CLAVE PRIVADA DE ALICE: a\n-------------------')
print(f'a = {alice.get_private_key()}\n')

print('\nCLAVE PÚBLICA DE ALICE: es el trío A=(g, p, g^a)\n-------------------')
print(f'g = {alice.g}\n')
print(f'p = {alice.p}\n')
print(f'g^a = {alice_pubkey}\n')

Claves de Alice:

- Clave privada es un solo número: $a_{priv}$. Este número es la clave privada de Alice y nunca saldrá del PC de Alice.
- Clave pública es un trío de números: $A_{pub}=\{g, p, g^a\}$. Esta es la clave pública de Alice, que es lo que le envía a Bob.

Nota: en realidad $g$ y $p$ no los suele escoger Alice sino que usa algún valor estándar de una tabla. Así que normalmente solo se envía $g^a$, y los valores de $g$ y $p$ son los del estándar. $g$ suele ser un primo pequeño, 2 ó 3, para acelerar los cálculos, mientras que $p$ es un número muy grande para dar seguridad al sistema.

### Bob

Bob calcular su par de claves públicas y privadas de una forma similar

In [ ]:
bob = DiffieHellman()
bob_pubkey = bob.gen_public_key()

print('CLAVE PRIVADA DE BOB: b\n-------------------')
print(f'b = {bob.get_private_key()}\n')

print('\nCLAVE PÚBLICA DE BOB: es el trío B=(g, p, g^b)\n-------------------')
print(f'g = {bob.g}\n')
print(f'p = {bob.p}\n')
print(f'g^b = {bob_pubkey}\n')

### Alice y Bob: cálculo de la clave compartida

Ahora (recuerda que todos estos cálculos son $\mod p$):

- Alice recibe de Bob $B=g^b$, y puede calcular la clave compartida usando su propia $a$: $k_{shared} = B^a=g^{ab}$
- Bob recibe de Alice $A=g^a$, y puede calcular la clave compartida usando su propia $b$: $k_{shared} = A^b=g^{ab}$
- Un espía de las comunicaciones sabe $A=g^a$ y $B=g^b$, pero no sabe $a$ ni $b$. Observa que sabiendo solo $g^a$ y $g^b$ no hay forma de calcular $g^{ab}$ y el atacante no puede calcular la clave

In [ ]:
alice_sharedkey = alice.gen_shared_key(bob_pubkey)
print(alice_sharedkey)

In [ ]:
bob_sharedkey = bob.gen_shared_key(alice_pubkey)
print(bob_sharedkey)
print(len(bob_sharedkey))

Y podemos comprobar que los dos tienen la misma clave compartida

In [ ]:
print(bob_sharedkey == alice_sharedkey)
print(f'Longitud de la clave: {len(bob_sharedkey) * 8} bits')

***

<font color="red">
Ya tenemos una "clave compartida", pero aún hay que adaptarla para poder usarla en AES-256 o ChaCha20. ¿Cómo lo harías?

Los parámetros p y g de la librería son muy antiguos (RFC3526). ¿Puedes buscar otros más modernos?
</font>

***

<font color="blue">
RESPUESTA: añade aquí tu respuesta
</font>

***

## RSA

### Generación de claves con RSA

Vamos a generar un par de claves pública/privada con RSA de tamaño 2048 bits


In [ ]:
# Clave de 2048 bits
from Crypto.PublicKey import RSA
key2048 = RSA.generate(2048)
key2048

***

<font color="red">
Mide cuánto tiempo necesitamos para generar una claves de un tamaño ya útil 2048.
    
Prueba a generar claves mayores, de 4096 bits y de 16384 bits, por ejemplo, que es similar en seguridad al cifrado AES de 256 bits. ¿Cuánto tiempo le lleva aproximadamente?
</font>

***

<font color="blue">
RESPUESTA
</font>


### Cifrado y descifrado

Recuerda de las transparencias que no es recomendable utilizar RSA "de forma pura", es decir, sin tener en cuenta muchas consideraciones sobre padding, conversiones, longitudes... que se recogen en [PKCS#1](https://en.wikipedia.org/wiki/PKCS_1). De hecho, PyCryptoDome no nos va a dejar utilizar el cifrado y descifrado directamente.

Observa que la línea siguiente da un error, avisando que uses el módulo  `Crypto.Cipher.PKCS1_OAEP`

In [ ]:
key2048.encrypt(b'hola', None)

**Aunque no se debe**, vamos a utilizar la función `_encrypt()` y `_decrypt()`, que no está documentada pero la puedes encontrar en el código: https://github.com/Legrandin/pycryptodome/blob/master/lib/Crypto/PublicKey/RSA.py#L178

Fíjate: vamos simplemente a cifrar el número '15' (que ocuparía 1 byte). Observa que el número '15' cifrado da un número enorme.

In [ ]:
m = 15
c = key2048._encrypt(m)
d = key2048._decrypt(c)
print(f"Cifrado: {c}")
print(f"Descifrado: {d}")

¿Cómo podemos cifrar una cadena como 'hola'? Pues, por ejemplo, cifrando los bytes de la cadena uno a uno. Observa todo lo que ocuparía esta solución.

In [ ]:
[key2048._encrypt(c) for c in b'hola']

Otra solución es convertir la cadena a un entero con la función: `msg = int.from_bytes(b"hola mundo", "big")`

In [ ]:
msg = int.from_bytes(b"hola mundo", "big")
print(msg)
key2048._encrypt(msg)

El problema de esto es que **no podemos cifrar un mensaje más grande que el tamaño en bits de la clave**, es decir, 2048 bits en este caso. Vamos a probar con "hola" * 1000, que es la cadena "hola" repetida 1000 veces

In [ ]:
key2048._encrypt(int.from_bytes(b"hola" * 1000, "big"))

Vamos a hacer las cosas bien: cifra `"hola mundo"` y `"hola mundo" * 1000` usando PKCS1 y nuestra clave RSA que hemos generado.

In [ ]:
from Crypto.Cipher import PKCS1_OAEP

cipher = PKCS1_OAEP.new(key2048)
c = cipher.encrypt(b'hola')
print(f'Cifrado RSA: {c}')
print(f'Cifrado: {cipher.decrypt(c)}')

c2 = cipher.encrypt(b'hola' * 1000)
print(f'Cifrado RSA: {c2}')
print(f'Cifrado: {cipher.decrypt(c2)}')



## Cifrado híbrido

Normalmente no se cifra un mensaje en RSA, sino que se usa algún tipo de cifrado híbrido:

- Cifrar con RSA la clave AES que se usa en las comunicaciones
- Cifrar con RSA el hash de un documento para firma digital

En el tema de TLS veremos un cifrado híbrido: ciframos con RSA la clave AES que usamos para cifrar el texto.

1. Bob: Crea par de claves RSA
1. Alice: Crea clave simétrica AES. Cifra la clave AES con la clave pública de Bob. Envía mensaje
1. Alice: cifra "hola mundo" con clave AES. Envía mensaje
1. Bob: descifra clave AES con clave privada. Descifra mensaje de Alice

Entre los ejemplos de RSA precisamente verás algo así: https://pycryptodome.readthedocs.io/en/latest/src/examples.html#encrypt-data-with-rsa

***

<font color="red">
¿Se te ocurre por qué es necesario el cifrado híbrido y no se cifra todo con RSA y ya está?
</font>

***

<font color="blue">
RESPUESTA
</font>